Run snakemake workflow
`snakemake --cores=12`

We obtained a .vcf file for our roommate data and 3 control .vcf files. Let's parse them.

In [61]:
import os
import pandas as pd

LIST_OF_FILES = ['SRR1705851_ref.SRR1705851.varscan_results.vcf',
                 'SRR1705851_ref.SRR1705858.varscan_results.vcf',
                 'SRR1705851_ref.SRR1705859.varscan_results.vcf',
                 'SRR1705851_ref.SRR1705860.varscan_results.vcf']


roommate_data = pd.read_csv(os.path.join('data', LIST_OF_FILES[0]), sep='\t', comment='#', header=None,
                            names=['CHROM', 'POS', 'ID','REF','ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'Sample1'])
roommate_data['FREQ'] = roommate_data['Sample1'].str.split(':').str.get(6).str.rstrip('%')
roommate_data['FREQ'] = pd.to_numeric(roommate_data['FREQ'])
roommate_data.drop(columns=['CHROM', 'ID', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'Sample1'], inplace=True)
print('Variants for roommate data')
display(roommate_data)

dfs = {}

pd.set_option('display.max_colwidth', None)
for file in LIST_OF_FILES[0:]: # all but the 0th (roommate's data)
    df = pd.read_csv(os.path.join('data', file), sep='\t', comment='#', header=None,
                     names=['CHROM', 'POS', 'ID','REF','ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'Sample1'])
    df['FREQ'] = df['Sample1'].str.split(':').str.get(6).str.rstrip('%')
    df['FREQ'] = pd.to_numeric(df['FREQ'])
    df = df[df['FREQ'] <= 90]
    df.drop(columns=['CHROM', 'ID', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'Sample1'], inplace=True)
    
    # print('Variants for', file)
    # display(df)
    df_mean = df['FREQ'].mean()
    print('Average FREQ:', round(df_mean, 5))
    df_std = df['FREQ'].std()
    print('Standart deviation for FREQ:', round(df_std, 5))
    print('Confidence margins:', round(df_mean - (df_std * 3), 5), '-', round(df_mean + (df_std * 3), 5))
    print('\n')
    dfs[file.split('.')[1]] = df
    # dfs[file.split('.')[1] + '_mean'] = df_mean
    # dfs[file.split('.')[1] + '_std'] = df_std
    # dfs[file.split('.')[1] + '_3std_range'] = round(df_mean - (df_std * 3), 5), '-', round(df_mean + (df_std * 3), 5)

    roommate_data = roommate_data.merge(df[['POS', 'FREQ']], how='left', on='POS', suffixes=(None, '_' + file.split('.')[1]))
roommate_data.drop(columns=['FREQ'], inplace=True)
roommate_data

Variants for roommate data


,POS,REF,ALT,FREQ
0,72,A,G,99.96
1,117,C,T,99.82
2,254,A,G,0.17
3,276,A,G,0.17
4,307,C,T,0.94
5,340,T,C,0.17
6,389,T,C,0.22
7,691,A,G,0.17
8,722,A,G,0.20
9,744,A,G,0.17


Average FREQ: 0.2775
Standart deviation for FREQ: 0.24065
Confidence margins: -0.44446 - 0.99946


Average FREQ: 0.25649
Standart deviation for FREQ: 0.07173
Confidence margins: 0.04131 - 0.47167


Average FREQ: 0.23692
Standart deviation for FREQ: 0.05238
Confidence margins: 0.07979 - 0.39405


Average FREQ: 0.25033
Standart deviation for FREQ: 0.07804
Confidence margins: 0.01621 - 0.48444




,POS,REF,ALT,FREQ_SRR1705851,FREQ_SRR1705858,FREQ_SRR1705859,FREQ_SRR1705860
0,72,A,G,NaN,0.30,NaN,NaN
1,117,C,T,NaN,0.30,NaN,NaN
2,254,A,G,0.17,0.25,0.19,0.23
3,276,A,G,0.17,0.22,0.24,0.33
4,307,C,T,0.94,NaN,NaN,NaN
5,340,T,C,0.17,0.23,0.21,0.21
6,389,T,C,0.22,0.23,NaN,0.20
7,691,A,G,0.17,0.23,0.23,0.23
8,722,A,G,0.20,0.23,0.23,0.25
9,744,A,G,0.17,0.21,0.25,0.22
